# Project Title
### Data Engineering Capstone Project

#### Project Summary
US immigration, demographics and temperature data is extracted, transformed and loaded into a Postgres DB to allow analysis of the us immigrations e.g. for touristic analysis.

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd

pd.set_option("display.max_columns", None)

import immigration
import demographics
import temperatures
import database
from queries import insert_into

In [2]:
database.reset_database()

# Step 0: General Info

The configuration for the project can be found in the `config.toml` file. Following settings can be defined there:

- postgres connection settings
- path to data files

The used PostgresDB runs locally inside a docker container, see the provided `docker-compose.yml`.

# Step 1: Scope the Project and Gather Data

### Scope 
Explain what you plan to do in the project in more detail.

- What data do you use?
    - Several datasets are used to provide a base for analysis of immigration into the US:
        - us immigration data
        - us demographics data (on city and state level)
        - international temperature data
- What is your end solution look like?       
    - After the extraction and transformation of the data, the data is loaded into a PostgresDB in a star schema (see the model below). The fact table consists of each immigrant. The dimensions table provide additional information, which allows to query the data to answer different questions:
        - How many immigrations were made per day/week/month per state depending on the visa type?
        - How does the amount of tourist immigrations vary with calendar month or temperature of target city/state?
        - How does the amount of immigrations from severel world regions depend on the demographics of the target area?
        - etc...
- What tools did you use?
    - Python functions were used for etl, a local PostgresDB inside a docker container was used as database (see the provided `docker-compose.yml`).

### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

- US immigration data (from US National Tourism and Trade Office)
    - each row describes a person immigrating inside the US
    - additional information about the person and the immigration are provided, e.g. the port of the immigration, the age of the person, etc.
    - many information in the raw immigration data is abbreviated by codes. An accompanying text files provides a key-value mapping for the codoes, e.g. port_code -> name of immigration port
- US demographics data by city (from OpenSoft https://public.opendatasoft.com/explore/dataset/us-cities-demographics/export/)
    - describes basic demographics for the bigger US cities
- US demographics data by state (from KFF https://www.kff.org/other/state-indicator/distribution-by-raceethnicity/?dataView=1&currentTimeframe=3)
    - describes basic demographics for all US states
- Internation temperature data (from Kaggle https://www.kaggle.com/berkeleyearth/climate-change-earth-surface-temperature-data)
    - describes the historic mean temperature for big international cities by city, year, month

# Step 2: Explore and Assess the Data
## Explore the Data and Cleaning Steps

The result of the data exploration and the performed cleaning steps are briefly described below. The used functions are documented in the source code.

### Immigration data

Based on part of the data (approx. 3 million records) the following observations were made:

#### Dropped columns

The following columns are not used:

- biryear (redundant because of i94bir)
- entdepu (high number of nulls, unknown meaning)
- entdepa (unknown meaning)
- entdepd (unknown meaning)
- count (no information)
- matflag (meaning unknown)
- occup (high number of nulls)
- visapost (high number of nulls, no relevant information for the case)
- insnum (high number of nulls, no information)
- dtadfile (no relevant information for the case)
- draddto (no relevant information for the case)
- visatype (no relevant information for the case)
- fltno (no relevant information for the case)
- admnum (no information)
- i94cit (a lot of codes - approx. 15 % - are not found in the regions list of the i94 description; in addition the exact meaning of the column is not known)

#### null values

- i94bir: <0.1% of values are NaN. Nulls cause problems with aggregation, therefore the field is imputed with the median
- i94mode: <0.01% of i94mode values are NaN, a value of 9 has the meaning "not reported", therefore the NaN values are filled with 9
- i94addr: approx 5% of values are NaN. While a value of '99' has the meaning "others", it cannot be used here as the reason why the high number of NAs are present is not known
- gender: approx 15% of values are NaN, leave as is
- airline: < 1%, leave as is
- depdate: approx 5%, leave as is, the probable meaning is, that the immigrant has not departed yet

i94addr, gender, airline, depdate are allowed to be nullable

#### Additional cleaning steps

- arrdate and depdate were converted from SAS date format
- depdate: pandas NaT values were converted to None as psycopg2 does not know how to handle pandas NaT format


### Accompanying immigration data

The accompanying text file (I94_SAS_Labels_Descriptions.sas) was parsed the extract the following mappings:
- port_code -> port
    - the city, country and the us state was extracted from the port name
- region_code -> region name
    - in most cases the region is a single country
- state_code -> state name
    - it was enriched by the us state demographics data (see below)
- visa_code -> kind of visa (e.g. business or pleasure)
- travel_mode_code -> travel mode (e.g. air or sed)

### Demographics data (city level)

- some columns were not relevant and were not used:
    - State (redundant)
    - Number of Veterans (not relevant for this use case)
    - Median Age (not relevant for this use case)
    - Average Household Size (not relevant for this use case)
    - Male Population (not relevant for this use case)
    - Female Population (not relevant for this use case)
- The data is provided partly as wide and partly as a long table. E.g. the total population has its own columns, however the race columns has multiple entries per city and race.
Therefore the race columns was pivoted. The long table is transformed into a wide table for the population numbers per race.

### Demographics data (state level)

The data was used as is.

### Temperature data

- For US states no state_code was provided. However, the gps coordinates were provided, they were used to distinguish between US cities with the same name.
- For very old timeframes (e.g. year 1900) many values were missing. However, only the temperature in the near past is relevant here.
- Therefore the only the last three values for each city and calendar month were used to calculate the mean temperature for each city and calendar month.

# Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

The model schema of the immigration table looks like the following:

![DB_Schema](schemaspy/results/diagrams/summary/relationships.real.large.png)

This model was chosen as the basic fact in this context is the immigration of one person, thus the _immigration_ table is defined as the fact table of the star schema. The other tables are dimensional tables, which provide additional information for the immigration process.

Two additional tables (demographics and temperatures) are not shown in the relationsship diagram as no foreign key relationsship can be formed between the immigration tables and this two tables. Of course, there are columns on which the tables can be joined (city in case of the temperatures table, and city+state_code in case of the demographics table), however, in both tables there are entries, which are not present in the other one.

![DB_Schema](schemaspy/results/diagrams/orphans/demographics.1degree.png)

![DB_Schema](schemaspy/results/diagrams/orphans/temperatures.1degree.png)

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model:
- loads the file with the accompanying immigration data, process it and save to the dimension tables
- extract immigration data, transform it and save to the _immigration_ fact table
- extract demographics data, transform it and save to the _demographics_ table
- extract temperatures data, transform it and save to the _temperatures_ table

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
The data model is created by etl functions in seperate python files. Please see the docstrings and the comments inside the files for details.

In [3]:
# loads the file with the accompanying immigration data, process it and save to the dimension tables
immigration.load_i94_data_to_db()

In [4]:
# extracts immigration data, transform it and save to the _immigration_ fact table
immigration_raw = immigration.load_immigration()
immigration_clean = immigration.clean_immigration(immigration_raw)
database.bulk_insert_data(insert_into.immigration, immigration_clean)

In [5]:
immigration_clean.head()

,i94yr,i94mon,i94res,i94port,i94mode,i94addr,arrdate,depdate,i94bir,i94visa,gender,airline
0,2016.0,4.0,692.0,XXX,9.0,None,2016-04-29,None,37.0,2.0,None,None
1,2016.0,4.0,276.0,ATL,1.0,AL,2016-04-07,None,25.0,3.0,M,None
2,2016.0,4.0,101.0,WAS,1.0,MI,2016-04-01,2016-08-25,55.0,2.0,M,OS
3,2016.0,4.0,101.0,NYC,1.0,MA,2016-04-01,2016-04-23,28.0,2.0,None,AA
4,2016.0,4.0,101.0,NYC,1.0,MA,2016-04-01,2016-04-23,4.0,2.0,None,AA


In [5]:
# extracts demographics data, transform it and save to the _demographics_ table
demographics_raw = demographics.load_demographics()
demographics_clean = demographics.clean_demographics(demographics_raw)
demographics_final = demographics.prepare_demographics_for_load(demographics_clean)
database.insert_data(insert_into.demographics, demographics_final)

In [7]:
demographics_clean.head()

,city,state_code,total_population,foreign_born,american_indian_and_alaska_native,asian,black_or_african_american,hispanic_or_latino,white
0,Silver Spring,MD,82463,30908.0,1084,8841,21330,25924,37756
1,Quincy,MA,93629,32935.0,351,30473,3917,2566,58723
2,Hoover,AL,84839,8229.0,0,4759,18191,3430,61869
3,Rancho Cucamonga,CA,175232,33878.0,2789,24519,24437,65823,111832
4,Newark,NJ,281913,86253.0,2268,7349,144961,100432,76402


In [6]:
# extracts temperatures data, transform it and save to the _temperatures_ table
temperatures_raw = temperatures.load_temperatures()
temperatures_clean = temperatures.clean_temperatures(temperatures_raw)
temperatures_agg = temperatures.aggregate_temperatures(temperatures_clean)
database.bulk_insert_data(insert_into.temperatures, temperatures_agg)

In [9]:
temperatures_agg.head(24)

,city,country,latitude,longitude,calendar_month,mean_temperature
0,A Coruña,Spain,42.59N,8.73W,1,9.381333
1,A Coruña,Spain,42.59N,8.73W,2,9.014667
2,A Coruña,Spain,42.59N,8.73W,3,11.564000
3,A Coruña,Spain,42.59N,8.73W,4,12.668667
4,A Coruña,Spain,42.59N,8.73W,5,14.931000
5,A Coruña,Spain,42.59N,8.73W,6,16.925667
6,A Coruña,Spain,42.59N,8.73W,7,19.060333
7,A Coruña,Spain,42.59N,8.73W,8,19.434667
8,A Coruña,Spain,42.59N,8.73W,9,18.595000
9,A Coruña,Spain,42.59N,8.73W,10,15.255667


#### 4.2 Data Quality Checks

To ensure data quality, the following constraints were created in the DDL queries:
- (compound) primary keys
- foreign key relationships
- data type checks
- not null checks

#### 4.3 Data dictionary 

see external [data dictionary markdown file](data_dictionary.md)


#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
    * The data size is medium, no fast access is needed, therefore no big data technologies like AWS Redshift, Apache Cassandra or Appache Spark were needed. Therefore a relational database like postgres is best for this case as it allows to include data quality checks in the definition of the tables.


* Propose how often the data should be updated and why.

    * Immigration data:
        * Monthly, as the data is only slowly changing. But this depends on the exact use case. In some cases a daily update would be benefical.
    * Temperature data:
        * Yearly, as the data is aggregated on a monthly level and is only slowly changing.
    * Demographics data:
        * Yearly, as the data is only slowly changing


* Write a description of how you would approach the problem differently under the following scenarios:
    * The data was increased by 100x.
        * Then a distributed system like Apache Spark is needed for the ETL process.
    * The data populates a dashboard that must be updated on a daily basis by 7am every day.
        * An Apache Airflow dag in an AWS fully managed airflow instance would be setup which starts the etl job on a daily basis e.g. on midnight.
    * The database needed to be accessed by 100+ people.
        * A big data database like AWS Redshift or Snowflake would be needed to store the data. Depending on the exact use case also Apache Cassandra would have to be used for predifined queries.
        
* Who is going to use this data model (user persona)?
    - Analysts at the US National Tourism and Trade Office or working for the tourism department of a city administration
    - Analysts for a hotel company, who wants to analyze where to open the next hotel and for which target group it should be designed
    - ...

* What are the types of questions this data model can answer?
    - How many immigrations were made per state depending on the visa type?
    - How many immigrations were made per state depending in relation to the state population size?
    - How many immigrations were made per day/week/month depending on the visa type?
    - How does the amount of tourist immigrations vary with calendar month or temperature of target city/state?
    - How does the amount of immigrations from several world regions depend on the demographics of the target area?
    - ...
    


### How many immigrations were made per state depending on the visa type?

In [22]:
query1 = """SELECT i94s.state, i94v.visa_name, COUNT(*) as cnt_immigrations
FROM immigration

INNER JOIN i94_states i94s on immigration.i94addr = i94s.state_code
INNER JOIN i94_visa i94v on i94v.visa_id = immigration.i94visa

WHERE state_code != '99'

GROUP BY i94s.state, i94v.visa_name
ORDER BY i94s.state, i94v.visa_name
"""

df1 = database.run_query(query1)
df1

,state,visa_name,cnt_immigrations
0,Alabama,Business,2770
1,Alabama,Pleasure,5271
2,Alabama,Student,147
3,Alaska,Business,299
4,Alaska,Pleasure,1287
...,...,...,...
156,Wisconsin,Pleasure,3965
157,Wisconsin,Student,323
158,Wyoming,Business,102
159,Wyoming,Pleasure,349


### How many immigrations were made per state depending in relation to the state population size?

This analysis shows very interesting results. While Hawaii gets the most immigrations in relation to its population size, there are states lates Mississippi or West Virginia, which hardly get any immigrations (in relation to the population size). Further analysis must be conducted to look for the reasons. One theory might be, that states with a very low immigration ratio don't have any big international airport.

In [24]:
query2 = """SELECT i94s.state, ROUND(COUNT(*) / (i94s.total * 1.0), 4) AS immigrations_per_state_population
FROM immigration

INNER JOIN i94_states i94s on immigration.i94addr = i94s.state_code

WHERE state_code != '99'
AND i94s.total IS NOT NULL

GROUP BY i94s.state, i94s.total
ORDER BY immigrations_per_state_population DESC
"""

df2 = database.run_query(query2)
df2.head(52)

,state,immigrations_per_state_population
0,Hawaii,0.1237
1,District of Columbia,0.0438
2,Nevada,0.0397
3,Florida,0.0309
4,New York,0.0288
5,Nebraska,0.0144
6,California,0.0122
7,Massachusetts,0.0107
8,New Jersey,0.0087
9,Washington,0.0078
